In [1]:
import sys
import os
parent_dir = os.path.abspath('../..')
sys.path.append(parent_dir)
import urdf2casadi.urdfparser as u2c
import urdf2casadi.geometry.plucker as pluck
import urdf2casadi.geometry.quaternion as quatT
import numpy as np
import casadi as ca

In [2]:
ur5 = u2c.URDFparser()
cwd = os.getcwd()
path_to_urdf = os.path.join(cwd, 'alpha_5_robot.urdf')
ur5.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

n_joints = ur5.get_n_joints(root, tip)
i_X_0fs = ur5.forward_kinematics(root, tip, floating_base = True)

q = ca.SX.sym("q", n_joints)
baseT_xyz = ca.SX.sym('T_xyz', 3) # manipulator-vehicle mount link xyz origin 
baseT_rpy = ca.SX.sym('T_rpy', 3) # manipulator-vehicle mount link rpy origin
base_T = ca.vertcat(baseT_rpy, baseT_xyz) # transform from origin to 1st child
x = ca.SX.sym('x')
y = ca.SX.sym('y')
z = ca.SX.sym('z')
tr_n = ca.vertcat(x, y, z) # x, y ,z of uv wrt to ned origin
thet = ca.SX.sym('thet')
phi = ca.SX.sym('phi')
psi = ca.SX.sym('psi')
eul = ca.vertcat(phi, thet, psi)  # NED euler angular velocity
p_n = ca.vertcat(tr_n, eul) # ned total states
n = ca.vertcat(p_n, q) #NED position

i_X_0s = i_X_0fs(q, tr_n, eul, baseT_xyz, baseT_rpy)

In [3]:
H0 , R0, p0 = pluck.spatial_to_homogeneous(i_X_0s[0])
T0 = ca.vertcat(p0, quatT.rotation_matrix_to_quaternion(R0, order='wxyz'))
T0_euler = ca.vertcat(p0, pluck.rotation_matrix_to_euler(R0, order='xyz'))
dIFF_KinJ0 = ca.jacobian(T0_euler, ca.vertcat(n))
dIFF2_KinJ0 = ca.jacobian(dIFF_KinJ0, ca.vertcat(n))
# dot_J0 = (dIFF2_KinJ0@arm_ss.dn).reshape((6, 10))

H1 , R1, p1 = pluck.spatial_to_homogeneous(i_X_0s[1])
T1 = ca.vertcat(p1, quatT.rotation_matrix_to_quaternion(R1, order='wxyz'))
T1_euler = ca.vertcat(p1, pluck.rotation_matrix_to_euler(R1, order='xyz'))
dIFF_KinJ1 = ca.jacobian(T1_euler, ca.vertcat(n))
dIFF2_KinJ1 = ca.jacobian(dIFF_KinJ1, ca.vertcat(n))
# dot_J1 = (dIFF2_KinJ1@arm_ss.dn).reshape((6, 10))

H2 , R2, p2 = pluck.spatial_to_homogeneous(i_X_0s[2])
T2 = ca.vertcat(p2, quatT.rotation_matrix_to_quaternion(R2, order='wxyz'))
T2_euler = ca.vertcat(p2, pluck.rotation_matrix_to_euler(R2, order='xyz'))
dIFF_KinJ2 = ca.jacobian(T2_euler, ca.vertcat(n))
dIFF2_KinJ2 = ca.jacobian(dIFF_KinJ2, ca.vertcat(n))
# dot_J2 = (dIFF2_KinJ2@arm_ss.dn).reshape((6, 10))

H3 , R3, p3 = pluck.spatial_to_homogeneous(i_X_0s[3])
T3 = ca.vertcat(p3, quatT.rotation_matrix_to_quaternion(R3, order='wxyz'))
T3_euler = ca.vertcat(p3, pluck.rotation_matrix_to_euler(R3, order='xyz'))
dIFF_KinJ3 = ca.jacobian(T3_euler, ca.vertcat(n))
dIFF2_KinJ3 = ca.jacobian(dIFF_KinJ3, ca.vertcat(n))
# dot_J3 = (dIFF2_KinJ3@arm_ss.dn).reshape((6, 10))

H4 , R4, p4 = pluck.spatial_to_homogeneous(i_X_0s[4])
T4 = ca.vertcat(p4, quatT.rotation_matrix_to_quaternion(R4, order='wxyz'))
T4_euler = ca.vertcat(p4, pluck.rotation_matrix_to_euler(R4, order='xyz'))
dIFF_KinJ4 = ca.jacobian(T4_euler, ca.vertcat(n))
dIFF2_KinJ4 = ca.jacobian(dIFF_KinJ4, ca.vertcat(n))
# dot_J4 = (dIFF2_KinJ4@arm_ss.dn).reshape((6, 10))



fk_eval = ca.Function("fkeval", [n, base_T], [T0, T1, T2, T3, T4])

# c , cpp or matlab code generation for forward dynamics
fk_eval.generate("fk_eval_.c")
os.system(f"gcc -fPIC -shared fk_eval_.c -o libFK.so")

0